# A Diet Problem
This notebook is based on a problem posed in Chapter 10 of the AIMMS documentation:<br>
https://documentation.aimms.com/_downloads/AIMMS_modeling.pdf

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

**Problem Description:**<br>
Consider the following table that includes 9 different menu items, nutritional facts associated with each menu item, max servings and prices.

In [3]:
menu_df = pd.DataFrame()
menu_df["item"] = ["Big Mac", "Quarter Pounder", "Vegetable Burger", "French Fries", "Salad", \
                  "Lowfat Milk", "Coca Cola", "Big Mac Menu", "Quarter Pounder Menu"]
menu_df["Calories"] = [479, 517, 341, 425, 54, 120, 184, 1202.4, 1240.4]
menu_df["Protein"] = [25, 32.4, 11.7, 5, 4, 9, 0, 31.3, 38.7]
menu_df["Fat"] = [22, 25, 10.6, 21, 2, 4, 0, 48.7, 51.7]
menu_df["Carbohydrates"] = [44, 40.4, 50, 54, 5, 12, 46, 158.5, 154.9]
menu_df["max servings"] = [2, 2, 2, 2, 2, 2, 2, 2, 2]
menu_df["price"] = [5.45, 4.95, 3.95, 1.95, 3.95, 1.75, 2.75, 8.95, 8.95]

menu_df

,item,Calories,Protein,Fat,Carbohydrates,max servings,price
0,Big Mac,479.0,25.0,22.0,44.0,2,5.45
1,Quarter Pounder,517.0,32.4,25.0,40.4,2,4.95
2,Vegetable Burger,341.0,11.7,10.6,50.0,2,3.95
3,French Fries,425.0,5.0,21.0,54.0,2,1.95
4,Salad,54.0,4.0,2.0,5.0,2,3.95
5,Lowfat Milk,120.0,9.0,4.0,12.0,2,1.75
6,Coca Cola,184.0,0.0,0.0,46.0,2,2.75
7,Big Mac Menu,1202.4,31.3,48.7,158.5,2,8.95
8,Quarter Pounder Menu,1240.4,38.7,51.7,154.9,2,8.95


The problem can be summarized as follows:<br>
Minimize:$\;\;\;\;\;\;\;\;$ total cost of the menu,<br>
Subject to:<br>
Minimize:
$$
\sum_{f}^{} p_{f}x_{f}
$$
Subject to:
$$
\underline{m}{n} \le \sum_{f}^{} v_{fn}x_{f} \le \overline{m}{n} \;\;\;\;\;\;\;\; \forall n
$$
$$
x{f} \in \{0 ... u_{f}\}, integer  \;\;\;\;\;\;\;\; \forall f
$$

In [5]:
model = ConcreteModel()

model.n = RangeSet(4)
model.f = RangeSet(len(menu_df))

m_mins = [3000,65,0,375]
m_maxs = [100000,100000,117,100000]
def min_req(model, n):
    return m_mins[n-1]
def max_req(model, n):
    return m_maxs[n-1]

#Parameters
model.m_min = Param(model.n, initialize=min_req, within=NonNegativeIntegers)
model.m_max = Param(model.n, initialize=max_req, within=NonNegativeIntegers)

def max_serving(model, f):
    return menu_df.loc[f-1,"max servings"]
model.u = Param(model.f, initialize=max_serving, within=NonNegativeIntegers)

def price(model, f):
    return menu_df.loc[f-1,"price"]
model.p = Param(model.f, initialize=price, within=Reals)

def nutrient(model, f, n):
    return menu_df.iloc[f-1,n]
model.v = Param(model.f, model.n, initialize=nutrient, within=Reals)

#Variables
model.x = Var(model.f, within=NonNegativeIntegers)

def servings(model, f):
    return model.x[f] <= model.u[f]
model.c1 = Constraint(model.f, rule=servings)

def min_req_rule(model, n):
    return sum(model.v[f,n]*model.x[f] for f in model.f) >= model.m_min[n]
model.c2 = Constraint(model.n, rule=min_req_rule)

def max_req_rule(model, n):
    return sum(model.v[f,n]*model.x[f] for f in model.f) <= model.m_max[n]
model.c3 = Constraint(model.n, rule=max_req_rule)

def rule_OF(model):
    return sum(model.p[f]*model.x[f] for f in model.f)
model.objective = Objective(rule=rule_OF, sense=minimize)

opt = SolverFactory('glpk')
results = opt.solve(model)

In [6]:
print("The total price of the optimal menu is: ", value(model.objective))
model.x.pprint()

The total price of the optimal menu is:  24.599999999999998
x : Size=9, Index=f
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      3 :     0 :   1.0 :  None : False : False : NonNegativeIntegers
      4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      5 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      6 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      7 :     0 :   1.0 :  None : False : False : NonNegativeIntegers
      8 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
      9 :     0 :   2.0 :  None : False : False : NonNegativeIntegers
